In [20]:
import re
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [1]:
#знакомьтесь франкенштейн

model_name_food = 'numblilbug/food-rubert-sentiment-model'
model_name_interior = 'numblilbug/interior-rubert-sentiment-model'
model_name_price = 'numblilbug/price-rubert-sentiment-model'
model_name_service = 'numblilbug/service-rubert-sentiment-model'
model_name_whole = 'numblilbug/service-rubert-sentiment-model'

food_tokenizer = AutoTokenizer.from_pretrained(model_name_food)
food_model = AutoModelForSequenceClassification.from_pretrained(model_name_food)

interior_tokenizer = AutoTokenizer.from_pretrained(model_name_interior)
interior_model = AutoModelForSequenceClassification.from_pretrained(model_name_interior)

price_tokenizer = AutoTokenizer.from_pretrained(model_name_price)
price_model = AutoModelForSequenceClassification.from_pretrained(model_name_price)

service_tokenizer = AutoTokenizer.from_pretrained(model_name_service)
service_model = AutoModelForSequenceClassification.from_pretrained(model_name_service)

whole_tokenizer = AutoTokenizer.from_pretrained(model_name_whole)
whole_model = AutoModelForSequenceClassification.from_pretrained(model_name_whole)


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [3]:
numbers = []
text_list = []

file_path = '/content/test_reviews.txt'

with open(file_path, 'r') as file:
    for line in file:
        parts = line.strip().split('\t', 1)
        numbers.append(int(parts[0]))
        text_list.append(parts[1])


In [22]:
def predict(text, tokenizer, model):
  tokenized_input = tokenizer(text, return_tensors='pt')

  with torch.no_grad():
      model_output = model(**tokenized_input)

  probabilities = torch.softmax(model_output.logits, dim=1).tolist()[0]
  predicted_class = max(range(len(probabilities)), key=probabilities.__getitem__)
  return predicted_class


In [24]:
food_preds = []
interior_preds = []
price_preds = []
service_preds = []
whole_preds = []
for text in text_list:
  food_preds.append(predict(text, food_tokenizer, food_model))
  interior_preds.append(predict(text, interior_tokenizer, interior_model))
  price_preds.append(predict(text, price_tokenizer, price_model))
  service_preds.append(predict(text, service_tokenizer, service_model))
  whole_preds.append(predict(text, whole_tokenizer, whole_model))


In [31]:
class_labels = {'positive': 0, 'negative': 1, 'both': 2, 'neutral': 3, 'absence': 4}

def map_numbers_to_labels(predictions):
    return [key for key, value in class_labels.items() if value in predictions]

food_labels = map_numbers_to_labels(food_preds)
interior_labels = map_numbers_to_labels(interior_preds)
price_labels = map_numbers_to_labels(price_preds)
service_labels = map_numbers_to_labels(service_preds)
whole_labels = map_numbers_to_labels(whole_preds)

In [34]:
output_file = 'test_cats_predictions.txt'

with open(output_file, 'w') as file:
    for number in numbers:
        for interior, food, price, whole, service in zip(interior_labels, food_labels, price_labels, whole_labels, service_labels):
            file.write(f"{number}\tInterior\t{interior}\n")
            file.write(f"{number}\tFood\t{food}\n")
            file.write(f"{number}\tPrice\t{price}\n")
            file.write(f"{number}\tWhole\t{whole}\n")
            file.write(f"{number}\tService\t{service}\n")